<a href="https://colab.research.google.com/github/kanishkjain1995/Learning-Tensorflow/blob/master/LearningTensorflow_(Ch_5_LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
batch_size = 128
embedding_size = 64
time_steps = 6
hidden_layer_size = 32
num_classes = 2
element_size = 1

In [0]:
digit_to_word = {1:"one", 2:"two", 3:"three", 4:"four", 5:"five", 6:"six", 7:"seven", 8:"eight", 9:"nine"}
digit_to_word[0] = "PAD"

In [0]:
even_sentences = []
odd_sentences = []
sequence_len = []
for i in range(10000):
  seq_len = np.random.choice(range(3,7))
  sequence_len.append(seq_len)
  even_indices = [np.random.choice(range(2,10,2)) for i in range(seq_len)] + [0]*(6-seq_len)
  odd_indices = [np.random.choice(range(1,10,2)) for i in range(seq_len)] + [0]*(6-seq_len)
  
  even_sentences.append(" ".join([digit_to_word[ind] for ind in even_indices]))
  odd_sentences.append(" ".join([digit_to_word[ind] for ind in odd_indices]))
  
data = even_sentences + odd_sentences
sequence_len*=2

In [0]:
word2index_map = {}
index = 0
for sent in data:
  for word in sent.lower().split():
    if word not in word2index_map:
      word2index_map[word]=index
      index+=1
index2word_map = {index: word for word, index in word2index_map.items()} 
vocabulary_size = len(index2word_map)

In [0]:
labels = [1]*10000 + [0]*10000
def one_hot(vec, num_class):
  n = len(vec)
  one_hot_labels = np.zeros((n, num_class))
  one_hot_labels[range(n), vec] = 1
  return one_hot_labels

In [0]:
data_indices = list(range(len(data)))
np.random.shuffle(data_indices)

data = np.array(data)[data_indices]
sequence_len = np.array(sequence_len)[data_indices]
labels = np.array(one_hot(labels, 2))[data_indices]

train_x = data[:10000]
train_y = labels[:10000]
train_seqlens = sequence_len[:10000]

test_x = data[10000:]
test_y = labels[10000:]
test_seqlens = sequence_len[10000:]

In [0]:
def get_sentence_batch(batch_size, data_x, data_y, data_seqlens):
  instance_indices = np.random.choice(range(len(data_x)), batch_size)
  x = [[word2index_map[word] for word in sent.lower().split()] for sent in data_x[instance_indices]]
  y = data_y[instance_indices]
  seqlens = data_seqlens[instance_indices]
  return x, y, seqlens

In [0]:
_inputs = tf.placeholder(tf.int32, shape=[batch_size, time_steps])
_labels = tf.placeholder(tf.float32, shape=[batch_size, num_classes])
_seqlens = tf.placeholder(tf.int32, shape=[batch_size])

In [0]:
weights = {
    "linear_layer" : tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], mean=0.0, stddev=0.01))
}
biases = {
    "linear_layer" : tf.Variable(tf.truncated_normal([num_classes], mean=0.0, stddev=0.01))
}

# *Single Layer LSTM*

In [0]:
with tf.name_scope("embeddings") as scope:
  embedding = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0, name="embedding"))
  embed = tf.nn.embedding_lookup(embedding, _inputs)

In [0]:
with tf.variable_scope("lstm"):
  lstm_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_layer_size, forget_bias=1.0)
  outputs, states = tf.nn.dynamic_rnn(lstm_cell, embed, sequence_length=_seqlens, dtype=tf.float32)
final_output = tf.matmul(states[1], weights['linear_layer']) + biases['linear_layer']

W0710 00:29:15.917137 140669355210624 deprecation.py:323] From <ipython-input-11-a873f012ec42>:2: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0710 00:29:15.920614 140669355210624 deprecation.py:323] From <ipython-input-11-a873f012ec42>:3: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0710 00:29:16.014219 140669355210624 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead

# *Multi Layer LSTM*

In [0]:
multi_embedding_size = 32
with tf.name_scope("embeddings") as scope:
  embedding = tf.Variable(tf.random_uniform([vocabulary_size, multi_embedding_size], -1.0, 1.0, name="embedding"))
  embed = tf.nn.embedding_lookup(embedding, _inputs)

In [0]:
num_layers = 2
with tf.variable_scope("lstm_multi"):
  lstm_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_layer_size, forget_bias=1.0)
  cell = tf.contrib.rnn.MultiRNNCell(cells=[lstm_cell]*num_layers, state_is_tuple=True)
  outputs, states = tf.nn.dynamic_rnn(cell, embed, sequence_length=_seqlens, dtype=tf.float32)
final_output = tf.matmul(states[num_layers-1][1], weights['linear_layer']) + biases['linear_layer']

W0710 04:59:24.943510 140651867142016 deprecation.py:323] From <ipython-input-12-215c3aac57c2>:3: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0710 04:59:25.940642 140651867142016 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0710 04:59:25.942426 140651867142016 deprecation.py:323] From <ipython-input-12-215c3aac57c2>:4: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for 

In [0]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=final_output, labels=_labels))

train_step = tf.train.RMSPropOptimizer(1e-3, 9e-1).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(final_output, 1), tf.argmax(_labels, 1))
accuracy = 100*tf.reduce_mean(tf.cast(correct_pred, tf.float32))

W0710 04:59:27.218790 140651867142016 deprecation.py:323] From <ipython-input-13-39db1b461d30>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W0710 04:59:27.622259 140651867142016 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  for i in range(1000):
    x_train, y_train, seq_train = get_sentence_batch(batch_size, train_x, train_y, train_seqlens)
    #print(x_train, y_train, seq_train)
    sess.run(train_step, feed_dict={_inputs:x_train, _labels:y_train, _seqlens:seq_train})
    if i%100 == 0:
      train_accuracy = sess.run(accuracy, feed_dict={_inputs:x_train, _labels:y_train, _seqlens:seq_train})
      print("step {:.4f}, train accuracy {:.2f}%".format(i, train_accuracy))
  for i in range(5):
    x_test, y_test, seq_test = get_sentence_batch(batch_size, test_x, test_y, test_seqlens)
    test_accuracy = sess.run(accuracy, feed_dict={_inputs:x_test, _labels:y_test, _seqlens:seq_test})
    print("step {:.4f}, test accuracy {:.2f}%".format(i, test_accuracy))
  output_example = sess.run([outputs],feed_dict={_inputs:x_test, _labels:y_test, _seqlens:seq_test})
  states_example = sess.run([states[1]],feed_dict={_inputs:x_test, _labels:y_test, _seqlens:seq_test})

step 0.0000, train accuracy 53.12%
step 100.0000, train accuracy 100.00%
step 200.0000, train accuracy 100.00%
step 300.0000, train accuracy 100.00%
step 400.0000, train accuracy 100.00%
step 500.0000, train accuracy 100.00%
step 600.0000, train accuracy 100.00%
step 700.0000, train accuracy 100.00%
step 800.0000, train accuracy 100.00%
step 900.0000, train accuracy 100.00%
step 0.0000, test accuracy 100.00%
step 1.0000, test accuracy 100.00%
step 2.0000, test accuracy 100.00%
step 3.0000, test accuracy 100.00%
step 4.0000, test accuracy 100.00%
